In [ ]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00


In [ ]:
#importar el csv de ndemandas
df=pd.read_csv("/content/demanda_dataset.csv",index_col="Unnamed: 0")
df.index=pd.to_datetime(df.index)

In [ ]:
# Manipulación y tratamiento de Datos
import numpy as np
import pandas as pd

# Visualización de datos
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Modelación Arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# Métrica de Evaluación
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn import metrics

# No presentar advertencias
import warnings
warnings.filterwarnings("ignore")

# Modelo Auto-Arima
from pmdarima import auto_arima

In [ ]:
#columnas sobre las que va a iterar el codigo
df.columns

Index(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL',
       'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'MI', 'MN', 'MO', 'NC', 'NE', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'TN', 'TX', 'UT', 'VA',
       'WA', 'WI'],
      dtype='object')

In [ ]:

# Crear un DataFrame vacío con índice de fechas para almacenar las predicciones
df_predictions = pd.DataFrame(index=pd.date_range(start='2023-08-31', end='2028-08-31', freq='M'))

# Iterar sobre las columnas del DataFrame original
for column in df.columns:
  # Crear un DataFrame temporal con la columna actual
  df1 = df[[column]]

  # Dividir los datos en entrenamiento y prueba
  train_data = df1.loc[df.index < "2020-12-31"]
  test_data = df1.loc[df.index >= "2020-12-31"]
  test = test_data.copy()

  # Ajustar un modelo ARIMA automático a los datos de entrenamiento
  modelo_auto = auto_arima(train_data, start_p=0, d=1, start_q=0,
                           max_p=7, max_d=3, max_q=7, start_P=0,
                           D=1, start_Q=0, max_P=2, max_D=1,
                           max_Q=2, m=12, seasonal=True,
                           error_action='warn', trace=True,
                           suppress_warnings=True, stepwise=True,
                           random_state=20, n_fits=50)

  # Obtener los parámetros del modelo ARIMA ajustado
  params = modelo_auto.get_params()
  order_param = params.get('order')  # Parámetros (p, d, q)
  seasonal_order_param = params.get('seasonal_order')  # Parámetros (P, D, Q, m)

  # Desempaquetar los parámetros
  p, d, q = order_param
  P, D, Q, m = seasonal_order_param

  # Ajustar un modelo SARIMA utilizando los parámetros obtenidos
  arima_model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, 12))
  arima_result = arima_model.fit()

  # Realizar las predicciones para las fechas futuras
  predictions = arima_result.predict(start='2023-08-31', end='2028-08-31', typ="levels")

  # Agregar las predicciones al DataFrame de resultados utilizando la columna actual como nombre de columna
  df_predictions[column] = predictions


In [ ]:

from google.colab import files

# Guardar el DataFrame modificado en un nuevo archivo CSV
df_predictions.to_csv('prediccion_demanda_arima.csv', index=False, encoding='utf-8-sig')

# Descargar el archivo CSV
files.download('prediccion_demanda_arima.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>